Script that 
* takes as input the files config-layers-X (X: polygon, point, linestring), and municipality codes
* for all municipality codes jointly, makes the "technical network"
* for each X,
    * determine evaluation layers
    * fetch and merge gdfs for given muni (if they exist)
    * save to `input-for-bike-node-planner/X`

In [1]:
# import libraries
import os
import yaml
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString

exec(open("../src/helper_functions.py").read())
exec(open("../src/tech-to-comm.py").read())

In [2]:
# read config files
config = yaml.load(
    open("../config.yml"), 
    Loader=yaml.FullLoader)
proj_crs = config["proj_crs"]
wfs_version = config["geofa_wfs_version"]
node_layer_name = config["geofa_nodes_layer_name"]
stretches_layer_name = config["geofa_stretches_layer_name"]


municipalities = yaml.load(
    open("../config-municipalities.yml"), 
    Loader=yaml.FullLoader)
codes = municipalities["kommunekode"]

geomtypes = ["point", "linestring", "polygon"]
config_layers = {}
for geomtype in geomtypes:
    config_layers[geomtype] = yaml.load(
        open(f"../config-layers-{geomtype}.yml"), 
        Loader=yaml.FullLoader)

In [3]:
# make folders
exec(open("./make_folders.py").read())

Folders created!


In [4]:
# remove previous output
remove_output_data(
    [
        "../input-for-bike-node-planner/dem",
        "../input-for-bike-node-planner/elevation",
        "../input-for-bike-node-planner/linestring/",
        "../input-for-bike-node-planner/network/",
        "../input-for-bike-node-planner/point/",
        "../input-for-bike-node-planner/polygon/",
        "../input-for-bike-node-planner/studyarea/"        
    ],
    remove_previous_output=True,
)

Data folder cleaned!


**Study area**

In [5]:
### read in municipality boundaries & create study area polygon
gdf = gpd.read_file("../data/municipality-boundaries/municipality-boundaries.gpkg")
gdf = gdf.to_crs(proj_crs) # make sure we have the right projected CRS
gdf = gdf[gdf["kommunekode"].isin(codes)] # filter to municipality codes indicated in config file
gdf_studyarea = gpd.GeoDataFrame(
    {
        "geometry": [gdf.unary_union]
    },
    crs = proj_crs
)
gdf_studyarea.to_file(
    filename = "../input-for-bike-node-planner/studyarea/studyarea.gpkg", 
    index = False)
print("Study area polygon created")
del gdf

Study area polygon created


**Network**

In [6]:
# Fetch input data from GeoFA (technical network)

url = f"https://geofa.geodanmark.dk/ows/fkg/fkg/?request=GetFeature&typename={node_layer_name}&service=WFS&version={wfs_version}"

knudepunkter = gpd.read_file(url)

url = f"https://geofa.geodanmark.dk/ows/fkg/fkg/?request=GetFeature&typename={stretches_layer_name}&service=WFS&version={wfs_version}"

straekninger = gpd.read_file(url)

assert len(knudepunkter) > 0, "No nodes found"
assert len(straekninger) > 0, "No stretches found"

In [24]:
### Cut technical network to study area extent

# new_codes = [int(code[1:]) for code in codes]

# nodes = knudepunkter[knudepunkter["kommunekode"].isin(new_codes)]
# edges = straekninger[straekninger["kommunekode"].isin(new_codes)]

# assert len(nodes) > 0, "No nodes found in study area"
# assert len(edges) > 0, "No stretches found in study area"

# ## Reproject to project CRS

# nodes = nodes.to_crs(proj_crs)
# edges = edges.to_crs(proj_crs)

# # save
# os.makedirs("../input-for-bike-node-planner/network/technical/", exist_ok=True)
# edges.to_file("../input-for-bike-node-planner/network/technical/edges_test.gpkg", index = False)
# nodes.to_file("../input-for-bike-node-planner/network/technical/nodes_test.gpkg", index = False)

# print("Technical nodes and edges for study area saved!")

Technical nodes and edges for study area saved!


In [23]:
# limit to extent of study area
assert straekninger.crs == gdf_studyarea.crs
assert knudepunkter.crs == gdf_studyarea.crs

edges_studyarea = straekninger.sjoin(gdf_studyarea, predicate="intersects").copy()
edges_studyarea.drop(columns=["index_right"], inplace=True)
nodes_studyarea = knudepunkter.clip(edges_studyarea.buffer(500).unary_union)

# remove empty geometries
edges_studyarea = edges_studyarea[edges_studyarea.geometry.notna()].reset_index(
    drop=True
)
nodes_studyarea = nodes_studyarea[nodes_studyarea.geometry.notna()].reset_index(
    drop=True
)

# assert there is one (and only one) LineString per edge geometry row
nodes_studyarea = nodes_studyarea.explode(index_parts=False).reset_index(drop=True)
assert all(nodes_studyarea.geometry.type == "Point")
assert all(nodes_studyarea.geometry.is_valid)

# assert there is one (and only one) Point per node geometry row
edges_studyarea = edges_studyarea.explode(index_parts=False).reset_index(drop=True)
assert all(edges_studyarea.geometry.type == "LineString")
assert all(edges_studyarea.geometry.is_valid)

# save
os.makedirs("../input-for-bike-node-planner/network/technical/", exist_ok=True)
edges_studyarea.to_file(
    "../input-for-bike-node-planner/network/technical/edges_test_test.gpkg", index=False
)
nodes_studyarea.to_file(
    "../input-for-bike-node-planner/network/technical/nodes_test_test.gpkg", index=False
)

print("Technical nodes and edges for study area saved!")

Technical nodes and edges for study area saved!


In [25]:
edges_studyarea["edge_id"] = edges_studyarea.id_cykelknudepunktsstraekning
assert len(edges_studyarea) == len(edges_studyarea["edge_id"].unique())

nodes_studyarea["node_id"] = nodes_studyarea.id_cykelknudepkt
assert len(nodes_studyarea) == len(nodes_studyarea["node_id"].unique())

processed_edges = assign_edges_start_end_nodes(edges_studyarea, nodes_studyarea)

processed_edges = order_edge_nodes(processed_edges)

processed_edges = find_parallel_edges(processed_edges)

assert len(processed_edges) == len(edges_studyarea)

processed_nodes = nodes_studyarea.loc[
    nodes_studyarea["node_id"].isin(processed_edges["u"])
    | nodes_studyarea["node_id"].isin(processed_edges["v"])
]

In [26]:
# save to files
os.makedirs("../input-for-bike-node-planner/network/processed/", exist_ok=True)
processed_nodes.to_file(
    "../input-for-bike-node-planner/network/processed/nodes.gpkg", index=False
)
processed_edges.to_file(
    "../input-for-bike-node-planner/network/processed/edges.gpkg", index=False
)
# edges_communication_parallel.to_file(
#     "../input-for-bike-node-planner/network/processed/edges_parallel.gpkg",
#     index=False,
# )

print("Processed nodes and edges for study area saved!")

Processed nodes and edges for study area saved!


In [27]:
# # convert technical to communication layer
# nodes_communication, edges_communication, edges_communication_parallel = technical_to_communication(
#     node_gdf = nodes_studyarea,
#     edge_gdf = edges_studyarea 
#     )

# # save to files
# os.makedirs("../input-for-bike-node-planner/network/communication/", exist_ok=True)
# nodes_communication.to_file(
#     "../input-for-bike-node-planner/network/communication/nodes.gpkg", 
#     index = False)
# edges_communication.to_file(
#     "../input-for-bike-node-planner/network/communication/edges.gpkg", 
#     index = False)
# edges_communication_parallel.to_file(
#     "../input-for-bike-node-planner/network/communication/edges_parallel.gpkg", 
#     index = False)

# print("Communication nodes and edges for study area saved!")

**Evaluation layers**

In [28]:
# create a dictionary of evaluation layers (based on config file inputs)

layer_dict = {}

for geomtype in geomtypes:

    layer_dict[geomtype] = {}

    # determine evaluation layers
    layers = []
    for v in config_layers[geomtype].values():
        layers += list(set(v.values()))
    layers = list(set(layers))
    
    # determine data sets that go into each layer

    # key is name of merged output layer, value is a dict 
    for layer in layers:
        layer_dict[geomtype][layer] = {}
    
    # adding data source as key to dictindict IF relevant to layer
    for datasource, vdict in config_layers[geomtype].items():
        for layer in (set(vdict.values())):
            layer_dict[geomtype][layer][datasource] = []
    
    for datasource, vdict in config_layers[geomtype].items():
        for k, v in vdict.items():
            layer_dict[geomtype][v][datasource] += [k]

for geomtype in geomtypes:
    if "ignore" in layer_dict[geomtype]:
        del layer_dict[geomtype]["ignore"]

print(layer_dict)

{'point': {'poi': {'land_besoegs': ['Museum', 'Forlystelsespark', 'Dyrepark, zoo', 'Dyrepark', 'Anden seværdighed', 'Frilandsmuseum', 'Aboret, blomsterpark o.l.'], 'land_fortid': ['Gravhøj', 'Vold eller voldsted', 'Langdysse', 'Jættestue', 'Fortidsminde/ruin', 'Skanse', 'Ruin', 'Batteri', 'Dysse', 'Runddysse', 'Andet historisk anlæg', 'Fæstningsanlæg', 'Oldtidsvej', 'Vikingeborg', 'Oldtidsminde'], 'land_landemaerke': ['Kirke', 'Fyrtårn', 'Vejrmølle', 'Herregård', 'Slot'], 'land_udflugt': ['Udsigtspunkt', 'Naturområde', 'Fugletårn/Udsigtstårn', 'Badestrand/-sted', 'Blå Flag strand', 'Badepunkt', 'Badevandsflag']}, 'facility': {'facilit_rasteplads': ['Bænk', 'Picnicbord', 'Legeplads', 'Borde-bænke/Picnic', 'Tørvejrsrum/Madpakkehus', 'Naturlegeplads', 'Vandlegeplads'], 'facilit_service': ['Toilet', 'Drikkevand', 'Cykelhandler', 'Vandpost', 'Cykeludlejere', 'Cykelservice', 'Cykelpumpestation/-servicepost']}, 'service': {'facilit_indkoeb': ['supermarked', 'bager', 'kiosk', 'slagter', 'Lokal

In [29]:
# for each layer type (point/linestring/polygon),
for geomtype in geomtypes:

    print(geomtype)
    
    # go through all evaluation layers for that geomtype... 
    for layername, datadict in layer_dict[geomtype].items():
    
        print("\t", layername)
        final_gdf = gpd.GeoDataFrame()
        
        # go through each data source for that evaluation layer...
        for k, v in datadict.items():

            gdf = gpd.GeoDataFrame()

            # and fetch it for each municipality
            for code in codes:
                print("\t \t", code, k)
                # for each code, check if file exists, if yes: read it in, if not empty: concatenate
                fp = f"../data/{geomtype}/{code}/{k}.gpkg"
                if os.path.exists(fp):
                    gdf_muni = gpd.read_file(fp)
                    if not gdf_muni.empty:
                        gdf = pd.concat(
                            [
                                gdf,
                                gdf_muni
                            ]
                        )

            # if at least one of the municipalities has data from this data source,
            # add it to final gdf
            if not gdf.empty:
                final_gdf = pd.concat(
                    [
                        final_gdf,
                        gdf[gdf["type"].isin(v)]
                    ]
                )

        # save evaluation layer to file, if not empty
        if not final_gdf.empty:
            final_gdf = final_gdf.reset_index(drop=True)
            final_gdf.to_file(
                f"../input-for-bike-node-planner/{geomtype}/{layername}.gpkg", 
                index = False
            )
            print("\t", f"{layername} saved")
        else:
            print("\t", f"No data found for layer {layername}")

point
	 poi
	 	 0320 land_besoegs
	 	 0336 land_besoegs
	 	 0430 land_besoegs
	 	 0450 land_besoegs
	 	 0461 land_besoegs
	 	 0320 land_fortid
	 	 0336 land_fortid
	 	 0430 land_fortid
	 	 0450 land_fortid
	 	 0461 land_fortid
	 	 0320 land_landemaerke
	 	 0336 land_landemaerke
	 	 0430 land_landemaerke
	 	 0450 land_landemaerke
	 	 0461 land_landemaerke
	 	 0320 land_udflugt
	 	 0336 land_udflugt
	 	 0430 land_udflugt
	 	 0450 land_udflugt
	 	 0461 land_udflugt
	 poi saved
	 facility
	 	 0320 facilit_rasteplads
	 	 0336 facilit_rasteplads
	 	 0430 facilit_rasteplads
	 	 0450 facilit_rasteplads
	 	 0461 facilit_rasteplads
	 	 0320 facilit_service
	 	 0336 facilit_service
	 	 0430 facilit_service
	 	 0450 facilit_service
	 	 0461 facilit_service
	 facility saved
	 service
	 	 0320 facilit_indkoeb
	 	 0336 facilit_indkoeb
	 	 0430 facilit_indkoeb
	 	 0450 facilit_indkoeb
	 	 0461 facilit_indkoeb
	 	 0320 facilit_overnatning
	 	 0336 facilit_overnatning
	 	 0430 facilit_overnatning
	 	 04

**Elevation**

In [30]:
exec(open("./prepare_elevation_data.py").read())

Data folder cleaned!
Elevation data downloaded! Starting to merge...


/opt/miniconda3/envs/knupu/lib/python3.9/site-packages/osgeo/gdal.py:287: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Elevation data ready!


In [31]:
print("All done! Now you can copy-paste \nall subfolders of '/input-for-bike-node-planner/' \ninto the '/data/input/' folder of the main repo")

All done! Now you can copy-paste 
all subfolders of '/input-for-bike-node-planner/' 
into the '/data/input/' folder of the main repo
